In [26]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [27]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [28]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [29]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [30]:
# pickle.dump(words,open('words.pkl','wb'))
# pickle.dump(classes,open('classes.pkl','wb'))

In [31]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [32]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [33]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\rishb\AppData\Local\Temp\ipykernel_25604\2460837776.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [34]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [35]:
from tensorflow.keras.optimizers import SGD

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [36]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created and saved")

Epoch 1/200
57/57 [==============================] - 1s 4ms/step - loss: 4.8557 - accuracy: 0.0071
Epoch 2/200
57/57 [==============================] - 0s 4ms/step - loss: 4.7913 - accuracy: 0.0424
Epoch 3/200
57/57 [==============================] - 0s 4ms/step - loss: 4.6782 - accuracy: 0.0459
Epoch 4/200
57/57 [==============================] - 0s 4ms/step - loss: 4.5786 - accuracy: 0.0530
Epoch 5/200
57/57 [==============================] - 0s 4ms/step - loss: 4.4626 - accuracy: 0.0389
Epoch 6/200
57/57 [==============================] - 0s 4ms/step - loss: 4.3301 - accuracy: 0.0742
Epoch 7/200
57/57 [==============================] - 0s 4ms/step - loss: 4.2343 - accuracy: 0.0848
Epoch 8/200
57/57 [==============================] - 0s 6ms/step - loss: 4.1235 - accuracy: 0.0813
Epoch 9/200
57/57 [==============================] - 0s 4ms/step - loss: 3.9345 - accuracy: 0.1343
Epoch 10/200
57/57 [==============================] - 0s 4ms/step - loss: 3.7814 - accuracy: 0.1519
Epoch 11/

57/57 [==============================] - 0s 8ms/step - loss: 0.9422 - accuracy: 0.7385
Epoch 84/200
57/57 [==============================] - 0s 6ms/step - loss: 0.9906 - accuracy: 0.7067
Epoch 85/200
57/57 [==============================] - 0s 4ms/step - loss: 0.8789 - accuracy: 0.7208
Epoch 86/200
57/57 [==============================] - 0s 4ms/step - loss: 0.8632 - accuracy: 0.7420
Epoch 87/200
57/57 [==============================] - 0s 4ms/step - loss: 0.8911 - accuracy: 0.7420
Epoch 88/200
57/57 [==============================] - 0s 4ms/step - loss: 0.8898 - accuracy: 0.7173
Epoch 89/200
57/57 [==============================] - 0s 4ms/step - loss: 0.9608 - accuracy: 0.7173
Epoch 90/200
57/57 [==============================] - 0s 4ms/step - loss: 0.7634 - accuracy: 0.7633
Epoch 91/200
57/57 [==============================] - 0s 4ms/step - loss: 0.9167 - accuracy: 0.7244
Epoch 92/200
57/57 [==============================] - 0s 4ms/step - loss: 0.8237 - accuracy: 0.7527
Epoch 93/200


57/57 [==============================] - 1s 9ms/step - loss: 0.6932 - accuracy: 0.8057
Epoch 165/200
57/57 [==============================] - 0s 8ms/step - loss: 0.6419 - accuracy: 0.8163
Epoch 166/200
57/57 [==============================] - 0s 7ms/step - loss: 0.8076 - accuracy: 0.7739
Epoch 167/200
57/57 [==============================] - 0s 7ms/step - loss: 0.8625 - accuracy: 0.7703
Epoch 168/200
57/57 [==============================] - 0s 5ms/step - loss: 0.7591 - accuracy: 0.7880
Epoch 169/200
57/57 [==============================] - 0s 4ms/step - loss: 0.7556 - accuracy: 0.7527
Epoch 170/200
57/57 [==============================] - 0s 5ms/step - loss: 0.7999 - accuracy: 0.7739
Epoch 171/200
57/57 [==============================] - 0s 5ms/step - loss: 0.6988 - accuracy: 0.7809
Epoch 172/200
57/57 [==============================] - 0s 5ms/step - loss: 0.7234 - accuracy: 0.7774
Epoch 173/200
57/57 [==============================] - 0s 5ms/step - loss: 0.7007 - accuracy: 0.8304
Epoc